In [13]:
!pip install transformers faiss-cpu PyPDF2 nltk openai python-dotenv --quiet

import os
import torch
import numpy as np
from PyPDF2 import PdfReader
from nltk.tokenize import sent_tokenize
from transformers import AutoTokenizer, AutoModel
from faiss import IndexFlatL2
from sklearn.metrics.pairwise import cosine_similarity
from openai import OpenAI
import nltk
import os
from dotenv import load_dotenv
from openai import OpenAI

# Carrega as variáveis do arquivo .env
load_dotenv() 

# 2. Busca o VALOR da variável de ambiente
api_key_valor = os.getenv("OPENAI_API_KEY")

# 3. Teste rápido: Se printar None, o script não achou o arquivo .env
if api_key_valor is None:
    print("Erro: Arquivo .env não encontrado ou variável OPENAI_API_KEY está vazia.")
else:
    # 4. Inicializa o cliente com o VALOR
    client = OpenAI(api_key=api_key_valor)

# Download necessário para o tokenizador de sentenças
nltk.download('punkt')
nltk.download('punkt_tab')



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [22]:
# Download necessário para o sent_tokenize
nltk.download('punkt')

# --- 1. CONFIGURAÇÃO DO MODELO E ÍNDICE ---
model_name = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
dimension = 384  
index = IndexFlatL2(dimension)


# --- 2. FUNÇÕES DE UTILIDADE ---
def read_pdf(file_path):
    if not os.path.exists(file_path):
        print(f"Aviso: Arquivo {file_path} não encontrado.")
        return ""
    pdf_reader = PdfReader(file_path)
    text = ""
    for page in pdf_reader.pages:
        content = page.extract_text()
        if content:
            text += content
    return text

def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    # Pooling: Usando a média (Mean Pooling) para melhor representação semântica
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.detach().numpy()

# --- 3. PROCESSAMENTO E INDEXAÇÃO ---
file_paths = ["documento1.pdf", "documento2.pdf"] # Substitua pelos seus arquivos reais
data = []
all_embeddings = []

print("Processando documentos e gerando embeddings...")

for file_path in file_paths:
    text = read_pdf(file_path)
    if not text: continue
        
    sentences = sent_tokenize(text)
    
    # Metadados fictícios baseados no seu exemplo
    if "documento2.pdf" in file_path:
        metadata = {"fonte": file_path, "autor": "Michael de Mello Oliveira", "categoria": "Geologia"}
    else:
        metadata = {"fonte": file_path, "autor": "Saionara Rodrigues Nogueira", "categoria": "Engenharia ambiental e Sanitária"}

    for sentence in sentences:
        if len(sentence.strip()) < 15: continue
        emb = get_embedding(sentence)
        all_embeddings.append(emb)
        data.append({"text": sentence, "metadata": metadata})

if all_embeddings:
    embeddings_array = np.vstack(all_embeddings).astype('float32')
    index.add(embeddings_array)
    print(f"Sucesso: {index.ntotal} sentenças indexadas.")

# --- 4. BUSCA, FILTRO E RERANKING ---
def search_and_filter(query, categoria=None, top_k=10):
    query_embedding = get_embedding(query).astype('float32')
    scores, indices = index.search(query_embedding, top_k)
    
    results = []
    for i, score in zip(indices[0], scores[0]):
        if i == -1 or i >= len(data): continue
        item = data[i]
        
        # Filtro de metadados
        if categoria and item["metadata"].get("categoria") != categoria:
            continue
            
        results.append({
            "text": item["text"], 
            "metadata": item["metadata"]
        })
    return results

def rerank(results, query):
    if not results: return []
    query_emb = get_embedding(query)
    for res in results:
        doc_emb = get_embedding(res["text"])
        res["rerank_score"] = float(cosine_similarity(query_emb, doc_emb)[0][0])
    
    results.sort(key=lambda x: x["rerank_score"], reverse=True)
    return results

# --- 5. INTEGRAÇÃO COM OPENAI (GERAÇÃO) ---
def responder_com_llm(pergunta, contextos):
    # Consolida os textos recuperados para enviar à LLM
    contexto_unificado = "\n".join([f"- {c['text']}" for c in contextos])
    
    prompt_sistema = """Você é um assistente especializado. 
    Responda à pergunta do usuário utilizando APENAS o contexto fornecido abaixo.
    Se a resposta não estiver no texto, diga que a informação não consta nos documentos."""
    
    prompt_usuario = f"CONTEXTO:\n{contexto_unificado}\n\nPERGUNTA: {pergunta}"

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": prompt_sistema},
                {"role": "user", "content": prompt_usuario}
            ],
            temperature=0
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Erro ao chamar OpenAI: {e}"

# --- 6. EXECUÇÃO FINAL ---
if __name__ == "__main__":
    minha_pergunta = "Qual o nome da profissional de engenharia sanitária mencionada?"
    
    print(f"\nBuscando: {minha_pergunta}")
    
    # Passo 1: Busca e Filtro
    achados = search_and_filter(minha_pergunta,categoria='Engenharia ambiental e Sanitária', top_k=5)
    
    # Passo 2: Reranking
    refinados = rerank(achados, minha_pergunta)
    
    if refinados:
        # Passo 3: Geração com OpenAI
        print(achados[0])
        print(achados[1])
        print("Gerando resposta final...")
        resposta = responder_com_llm(minha_pergunta, refinados)
        print("-" * 30)
        print(f"RESPOSTA: {resposta}")
        print("-" * 30)
    else:
        print("Nenhum documento relevante encontrado.")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Processando documentos e gerando embeddings...
Sucesso: 13 sentenças indexadas.

Buscando: Qual o nome da profissional de engenharia sanitária mencionada?
{'text': 'Saionara Rodrigues Nogueira\nTipo\nProfissional\nRegistro\n205110-8\nEspecialidade\nEngenheiro de Segurança do Trabalho\nFormação Acadêmica\nEngenharia ambiental e Sanitária\nExperiência Profissional\nProfissional com experiência em mineração e setor industrial, com atuação em produção, qualidade,  \nmeio ambiente e segurança do trabalho.', 'metadata': {'fonte': 'documento1.pdf', 'autor': 'Saionara Rodrigues Nogueira', 'categoria': 'Engenharia ambiental e Sanitária'}, 'rerank_score': 0.6127815842628479}
{'text': 'Tenho uma abordagem analítica e estratégica para a resolução de problemas e melhoria  \ncontínua, o que me permite implementar soluções inovadoras que aumentam a eficiência \noperacional e asseguram a conformidade com os padrões de qualidade e regulamentações \nambientais e segurança.', 'metadata': {'fonte': 'docum